# Semantic Kernel Tutorial

This notebook demonstrates the key features of Semantic Kernel with Ollama.

## Prerequisites
- Ollama running locally on port 11434
- llama3.2 model pulled
- semantic-kernel Python package installed

## 1. Basic Setup

First, let's import the necessary modules and create a kernel with Ollama.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.contents import ChatHistory

# Create kernel and add Ollama service
kernel = Kernel()
service = OllamaChatCompletion(
    ai_model_id="llama3.2",
    host="http://localhost:11434",
)
kernel.add_service(service)

print("✅ Kernel initialized with Ollama service")

## 2. Simple Chat Completion

Let's send a simple message and get a response.

In [ ]:
chat_history = ChatHistory()
chat_history.add_user_message("Explain what Semantic Kernel is in one sentence.")

response = await service.get_chat_message_contents(
    chat_history=chat_history,
    settings=service.get_prompt_execution_settings_class()(
        max_tokens=100,
        temperature=0.7
    )
)

print(f"Response: {response[0].content}")

## 3. Creating a Custom Plugin

Plugins allow you to extend the AI's capabilities with custom functions.

In [ ]:
from typing import Annotated
from semantic_kernel.functions import kernel_function

class CalculatorPlugin:
    """A simple calculator plugin."""
    
    @kernel_function(
        name="add",
        description="Adds two numbers together"
    )
    def add(
        self,
        a: Annotated[float, "First number"],
        b: Annotated[float, "Second number"]
    ) -> Annotated[float, "The sum"]:
        return a + b
    
    @kernel_function(
        name="multiply",
        description="Multiplies two numbers"
    )
    def multiply(
        self,
        a: Annotated[float, "First number"],
        b: Annotated[float, "Second number"]
    ) -> Annotated[float, "The product"]:
        return a * b

# Add plugin to kernel
kernel.add_plugin(CalculatorPlugin(), plugin_name="calculator")
print("✅ Calculator plugin added")

## 4. Using Agents with Function Calling

Agents can automatically call functions when needed.

In [ ]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior

# Create agent with function calling
agent = ChatCompletionAgent(
    kernel=kernel,
    name="MathBot",
    instructions="You are a math assistant. Use the calculator functions when needed.",
    function_choice_behavior=FunctionChoiceBehavior.Auto(),
)

print("✅ Agent created with function calling enabled")

In [ ]:
# Test the agent
question = "What is 25 multiplied by 4, then add 10?"
print(f"Question: {question}\n")

response = await agent.get_response(messages=question)
print(f"Answer: {response.message}")

## 5. Multi-turn Conversation

Maintain context across multiple exchanges.

In [ ]:
chat = ChatHistory()

# Turn 1
chat.add_user_message("I'm planning a trip to Paris.")
response1 = await service.get_chat_message_contents(
    chat_history=chat,
    settings=service.get_prompt_execution_settings_class()(max_tokens=80)
)
chat.add_assistant_message(response1[0].content)
print(f"User: I'm planning a trip to Paris.")
print(f"AI: {response1[0].content}\n")

# Turn 2
chat.add_user_message("What city was I talking about?")
response2 = await service.get_chat_message_contents(
    chat_history=chat,
    settings=service.get_prompt_execution_settings_class()(max_tokens=50)
)
print(f"User: What city was I talking about?")
print(f"AI: {response2[0].content}")

## 6. Streaming Responses

Get responses as they're generated for a better user experience.

In [ ]:
from IPython.display import display, Markdown
import asyncio

chat_history = ChatHistory()
chat_history.add_user_message("Write a short poem about AI.")

print("Streaming response:\n")
full_response = ""

async for chunk in service.get_streaming_chat_message_contents(
    chat_history=chat_history,
    settings=service.get_prompt_execution_settings_class()(max_tokens=150)
):
    if chunk:
        for content in chunk:
            if content.content:
                full_response += content.content
                print(content.content, end="", flush=True)

print("\n\nDone!")

## Summary

You've learned:
- ✅ How to set up Semantic Kernel with Ollama
- ✅ How to send chat completions
- ✅ How to create custom plugins
- ✅ How to use agents with function calling
- ✅ How to maintain conversation context
- ✅ How to stream responses

Explore more at: https://github.com/microsoft/semantic-kernel